In [1]:
import json
import re
import nltk
import requests
from nltk.corpus import stopwords 
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.stem import WordNetLemmatizer
from sklearn.pipeline import make_pipeline, make_union
from sklearn.preprocessing import FunctionTransformer, LabelEncoder
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer

In [6]:
p.keys()

dict_keys(['category', 'type', 'publisher', 'author', 'institution', 'year', 'authorWords', 'institutionWords', 'textWords', 'titleWords', 'publisherWords', 'id', 'authorInstitution', 'catesearch'])

In [8]:
p['catesearch']['researcharticle']

['8',
 '9',
 '10',
 '11',
 '19',
 '20',
 '21',
 '28',
 '34',
 '35',
 '36',
 '37',
 '43',
 '44',
 '50',
 '51',
 '52',
 '53',
 '64',
 '65',
 '66',
 '67',
 '68',
 '69',
 '89',
 '90',
 '91',
 '98',
 '99',
 '105',
 '106',
 '112',
 '113',
 '124',
 '125',
 '133',
 '134',
 '142',
 '143',
 '151',
 '161',
 '167',
 '168',
 '173',
 '174',
 '182',
 '183',
 '191',
 '192',
 '202',
 '203',
 '204',
 '205',
 '211',
 '212',
 '219',
 '220',
 '231',
 '232',
 '233',
 '234',
 '245',
 '246',
 '253',
 '266',
 '267',
 '268',
 '269',
 '277',
 '278',
 '279',
 '287',
 '288',
 '289',
 '294',
 '295',
 '302',
 '303',
 '304',
 '312',
 '313',
 '314',
 '315',
 '316',
 '317',
 '326',
 '327',
 '328',
 '329',
 '337',
 '346',
 '347',
 '351',
 '352',
 '353',
 '360',
 '361',
 '362',
 '363',
 '364',
 '369',
 '376',
 '377',
 '378',
 '379',
 '392',
 '400',
 '401',
 '402',
 '403',
 '404',
 '409',
 '410',
 '419',
 '430',
 '431',
 '440',
 '441',
 '442',
 '450',
 '451',
 '452',
 '462',
 '463',
 '473',
 '474',
 '475',
 '476',
 '477',

dict_keys(['category', 'type', 'publisher', 'author', 'institution', 'year', 'authorWords', 'institutionWords', 'textWords', 'titleWords', 'publisherWords', 'id', 'authorInstitution', 'catesearch'])

In [2]:
with open("./data/json/metadata.json",'r',encoding='utf-8') as json_file:
    p = json.load(json_file)

In [162]:
g= {}
for c in p['id']:
    key = p['id'][c]['type'].replace(' ','').lower()
    if key not in g:
        g[key] = [c]
    else:
        g[key].append(c)

In [165]:
with open('./data/json/catesearch.json','w',encoding='utf-8') as json_file:
    json.dump(g, json_file)

In [15]:
path = get_tmpfile("word2vec.model")

model = Word2Vec(common_texts, size=100, window=5, min_count=1, workers=4)
model.save("word2vec.model")

In [6]:
sentences = [p['id'][key]['text'] for key in p['id']]

In [59]:
words = []
lemmatizer = WordNetLemmatizer()
for s in sentences:
    li = []
    for w in re.sub('[^a-z]',' ',s.lower()).split():
        w = lemmatizer.lemmatize(w)
        if w not in sw:
            li.append(w)
    words.append(li)
        

In [114]:
model = Word2Vec(words, sg=1, size=100, window=5, min_count=5, negative=3, sample=0.001, hs=1, workers=4)

In [115]:
model.train(words, total_examples=1, epochs=1)

(4351690, 4506165)

In [120]:
len(model.wv['weekly'])

100

In [128]:
v = np.zeros([len(words),100])
len_tfidf = vec.shape[1]

for i in range(vec.shape[0]):
    for j in range(vec.shape[1]):
        n = vec[i,j]
        if n > 0:
            w = features[j]
            w = lemmatizer.lemmatize(w)
            if w in model.wv:
                v[i,:] += n*model.wv[w]
    print('%04d \r'%i,end='')

In [130]:
v.shape

(3955, 100)

In [109]:
vectorizer = TfidfVectorizer(sublinear_tf=True, stop_words = 'english')
vec = vectorizer.fit_transform([' '.join(w) for w in words])

In [100]:
print(vec[2,3])

0.0


In [90]:
t = []
for i in vec[0,:]:
    t.append(i)

In [96]:
t[0]

<1x83645 sparse matrix of type '<class 'numpy.float16'>'
	with 316 stored elements in Compressed Sparse Row format>

In [111]:
features = vectorizer.get_feature_names()

In [112]:
fdict = {}
for i,f in enumerate(features):
    fdict[f] = i

In [139]:
from sklearn.neighbors import NearestNeighbors
nbrs = NearestNeighbors(n_neighbors=11, algorithm='ball_tree').fit(v)
distances, indices = nbrs.kneighbors(v)

In [140]:
indices[0,1:]

array([1992, 3855, 1515, 2600, 2579,  559,  784,  855,  163, 3785])

In [135]:
nbrs = NearestNeighbors(n_neighbors=11, algorithm='ball_tree').fit(vec)
distances, indices = nbrs.kneighbors(vec)

/Users/wangyuhan/anaconda/lib/python3.6/site-packages/sklearn/neighbors/base.py:216: UserWarning: cannot use tree with sparse input: using brute force
  warnings.warn("cannot use tree with sparse input: "


In [137]:
indices[0,1:]

array([3015, 2977, 2979, 2985, 2989, 3006, 3020, 2600, 1627,  493])

In [141]:
p['id']['3015']

{'author': ['Ilya Kapovich'],
 'authorInstitution': ['Department of Mathematics, University of Illinois at Urbana-Champaign, Urbana, IL 61801, USA.'],
 'category': 'Letters',
 'date': '2016/08/19',
 'doi': '10.1126/science.aah4571',
 'imageLink': None,
 'pdfLink': 'http://science.sciencemag.org/content/353/6301/758.2.full.pdf',
 'pptLink': '/content/353/6301/758.2.full.txt',
 'publisher': 'American Association for the Advancement of Science',
 'text': '',
 'textLink': '/content/353/6301/758.2.full.txt',
 'title': 'Open-access policies: A legal quagmire',
 'type': 'Letter'}

In [143]:
p['id']['0']

{'author': [],
 'authorInstitution': [],
 'category': 'None',
 'date': '2010/01/01',
 'doi': '10.1126/science.327.5961.100-a',
 'pdfLink': 'http://science.sciencemag.org/content/327/5961/100.1.full.pdf',
 'publisher': 'American Association for the Advancement of Science',
 'text': 'The MaxDiscovery Toxicity Assay Kits monitor in vivo toxicity by measuring liver and heart function in serum samples. These simple and convenient tests allow the rapid, colorimetirc determination of well-characterized functional markers of liver and heart toxicity. They can be used to monitor toxic effects caused by administration of small-interfering RNA, DNA plasmids, transfection agents, or other therapeutic treatments during preclinical studies. The assays can be used in a range of mammals, including mice and rats, providing a powerful tool to assess and reduce the toxicity of drug formulations and to improve drug delivery methods.  Bioo Scientific  For info: 512-707-8993 www.biooscientific.com The Prote